In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import os
from pytorch import *


In [2]:
with open("training.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()




In [3]:
x = torch.Tensor(x)
y = torch.Tensor(y)
train = TensorDataset(x,y)
train_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [4]:
input_dim = 6
output_dim = 2
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('lstm', model_params)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()

predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

[1/100] Training loss: 393.6323	
[2/100] Training loss: 201.2256	
[3/100] Training loss: 178.0359	
[4/100] Training loss: 165.9100	
[5/100] Training loss: 148.8695	
[6/100] Training loss: 124.9798	
[7/100] Training loss: 107.7048	
[8/100] Training loss: 99.5471	
[9/100] Training loss: 96.8886	
[10/100] Training loss: 91.2669	
[50/100] Training loss: 81.7700	
[100/100] Training loss: 94.5421	


NameError: name 'model_path' is not defined

In [24]:
torch.save(model.state_dict(), "LSTM_yes.pth")


AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'